## Load Dataset & Libraries

In [18]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('data.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_8120\477681342.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


## Preprocessing

In [33]:
df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,False,strict,Private room,2020.0,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,False,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,True,flexible,Private room,2005.0,$620,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and..."
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,True,moderate,Entire home/apt,2005.0,$368,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,False,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."


### Data Cleaning

First i will check if there is any missing or duplicate value in the dataframe

In [20]:
df.duplicated().sum()

541

I will drop the duplicate values to prevent some bias in the later tests

In [21]:
df = df.drop_duplicates()

Now i will move on to do a missing values verification, and if necessary, missing imputation

In [ ]:
percentage_result = df.isnull().sum()/df.shape[0] * 100

percentage_result = pd.DataFrame({"columns": percentage_result.keys(), "%": percentage_result})
percentage_result.reset_index(drop=True, inplace=True)
percentage_result

,columns,%
0,id,0.000000
1,NAME,0.244959
2,host id,0.000000
3,host_identity_verified,0.283172
4,host name,0.395853
5,neighbourhood group,0.028415
6,neighbourhood,0.015677
7,lat,0.007839
8,long,0.007839
9,country,0.521272


Almost all the rows have a small amonunt of missing, bolow 1%, but last review, reviews per month anf house rules have a 15-50% missing ratio
\
The frist two seems to be NMAR, since new hosts with 0 reviews can be the case of missing, i will test that below


In [ ]:
check_cols = ["last review", "reviews per month"]

for col in check_cols:
    missing_in_col = df[col].isna()
    zero_reviews = df["number of reviews"] == 0
    
    ratio = (missing_in_col & zero_reviews).sum() / missing_in_col.sum()
    print(f"Ratio of missing {col} in listings with 0 reviews: {ratio:.2f}")

Ratio of missing last review in listings with 0 reviews: 0.99
Ratio of missing reviews per month in listings with 0 reviews: 0.99
Ratio of missing review rate number in listings with 0 reviews: 0.10


Since the ratio is almost at 1005, i will fill the missings with 0

In [40]:
cols_to_fill_zero = ["reviews per month", "last review"]

df[cols_to_fill_zero] = df[cols_to_fill_zero].fillna(0)

In [26]:
#i will drop the license row since the missing is almost at 100%
df = df.drop(columns=["license"])

Now i will check if some columns have a high ratio of unique values, those will be: host id, host name, country and country code

In [ ]:
cols_to_check = ['host id', 'host name', 'country', 'country code']

unique_ratios = df[cols_to_check].nunique() / len(df)

print(unique_ratios)

host id         0.99999
host name       0.12924
country         0.00001
country code    0.00001
dtype: float64


Country and country code have a high unique value ratio, so i will drop them. This is expected since the dataframe only contains data from NYC. I will do the same with host id

In [ ]:
df = df.drop(columns=["country", "country code", "host id"])

For the house_rules row i will fill the missing with "no rules"

In [42]:
df["house_rules"] = df["house_rules"].fillna("No rules")

### Feature Engineering

Next steps:
- Create some columns that verifies if a airbnb has smoking, pet, party etc. rules
- Create a feature to measure the distance to the nearest subway station
- The same bu for the central park